# Récupération de données via ThingSpeak
Le programme vu dans le [notebook sur le dictionnaire de sons](dictionnaire_son.ipynb) nécessite la récupération de données de capteur *via* le service ThingSpeak. On essaiera dans ce notebook de récupérer des données depuis ThingSpeak afin de les utiliser dans un programme Python.

## Qu'est-ce que ThingSpeak ?
ThingSpeak est une plateforme permettant de stocker des données IoT (*Internet of Things*, &laquo;Internet des Objets&raquo;), et de proposer des outils d'analyse de données, pouvant être utilisés dans des environnements comme Matlab par exemple.
Nous allons ici essayer de récupérer des données de ThingSpeak vers Python.

## Mise en place de l'environnement d'utilisation.
On utilisera un channel ThingSpeak qui a déjà été créé :

[Graphique page web](https://thingspeak.com/channels/768391/charts/1?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15)

On a inséré trois points de données, de valeur 1, 4 et 10.
À chaque chaîne Thingspeak est associé une clé de lecture et une clé d'écriture. Comme on cherche ici à lire la chaîne Thingspeak, on utilisera la clé de lecture.

L'URL de l'API Thingspeak permettant de récupérer les d'un champ de données est défini comme qui suit :

![Structure des URL de lecture de champs ThingSpeak](img/url_ts.svg)

<center>Stucture d'un URL de lecture de champ ThingSpeak.</center>

## Application Python
On souhaite récupérer les données depuis le channel ThingSpeak. Pour utiliser les URL, on utilisera le module `urllib` de Python :

In [1]:
import urllib

On récupère ensuite les données depuis ThingSpeak, avec l'URL de lecture de champ, en utilisant la méthode `urllib.request.urlopen` :

In [2]:
tsField = urllib.request.urlopen('https://api.thingspeak.com/channels/768391/fields/1.json?api_key=ADAXKK2DQ0I5T9O4&results=3')

On obtient un objet `http.client.HTTPResponse`. On lit les données contenues par cet objet avec sa méthode `read` :

In [3]:
tsRawData = tsField.read()

La méthode `http.client.HTTPResponse.read` renvoie un objet de type `bytes`, qui correspond à une chaîne de données binaires brutes. On convertit donc ces données en chaîne de caractère avec la méthode `bytes.decode` :

In [4]:
tsTextData = tsRawData.decode('utf-8') # On encodera en chaîne de caractères suivant l'encodage UTF-8.

On obtiendra des données sous cette forme :

In [5]:
tsTextData

'{"channel":{"id":768391,"name":"Datatest","latitude":"0.0","longitude":"0.0","field1":"Field Label 1","created_at":"2019-04-27T15:37:44Z","updated_at":"2019-04-27T15:37:44Z","last_entry_id":3},"feeds":[{"created_at":"2019-04-28T15:07:49Z","entry_id":1,"field1":"01"},{"created_at":"2019-04-28T15:10:03Z","entry_id":2,"field1":"04"},{"created_at":"2019-04-28T15:10:39Z","entry_id":3,"field1":"010"}]}'

Il s'agit de données JSON (JavaScript Object Notation), un langage de description de données utilisant la notation objet du langage Javascript. On utilisera pour exploiter ces données le module `json` :

In [6]:
import json

On convertit les données JSON en données Python avec la méthode `json.loads` :

In [7]:
tsPythonData = json.loads(tsTextData)

On obtient :

In [8]:
tsPythonData

{'channel': {'id': 768391,
  'name': 'Datatest',
  'latitude': '0.0',
  'longitude': '0.0',
  'field1': 'Field Label 1',
  'created_at': '2019-04-27T15:37:44Z',
  'updated_at': '2019-04-27T15:37:44Z',
  'last_entry_id': 3},
 'feeds': [{'created_at': '2019-04-28T15:07:49Z',
   'entry_id': 1,
   'field1': '01'},
  {'created_at': '2019-04-28T15:10:03Z', 'entry_id': 2, 'field1': '04'},
  {'created_at': '2019-04-28T15:10:39Z', 'entry_id': 3, 'field1': '010'}]}

Les données sont donc contenues dans un dictionnaire. On remarque notamment que les données recherchées sont stockées dans des dictionnaires (clé `field1`), eux-même dans une liste, elle-même dans le dictionnaire principal.
On récupère donc les données comme qui suit :

In [9]:
for f in tsPythonData['feeds']:
    print('Entrée {0} | {1}'.format(f['entry_id'], f['field1'])) # Numéro de l'entrée, valeur.

Entrée 1 | 01
Entrée 2 | 04
Entrée 3 | 010


On retrouve donc bien les points de données correspondants.